<a href="https://colab.research.google.com/github/alesaccoia/DDM_2425/blob/main/AI_003_RAG_Gemini_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Esempio di RAG e di generazione del testo con Gemini Flash

In [1]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import google.generativeai as genai
import os

# Knowledge base con frasi strutturate che menzionano sempre la regione
knowledge_base = [
    # Toscana
    "La bistecca alla fiorentina è un piatto tipico della regione Toscana",
    "La ribollita è una zuppa tradizionale della regione Toscana",
    "Il cacciucco è una zuppa di pesce originaria della città di Livorno, in Toscana",
    "I cantucci sono biscotti alle mandorle tradizionali della regione Toscana",
    "Il pecorino di Pienza è un formaggio DOP prodotto in Toscana",
    "Il Brunello è un vino prodotto a Montalcino, in Toscana",

    # Sicilia
    "Gli arancini sono palle di riso ripiene tipiche della regione Sicilia",
    "La pasta alla Norma è un piatto tradizionale della città di Catania, in Sicilia",
    "I cannoli ripieni di ricotta sono dolci tipici della regione Sicilia",
    "La caponata di melanzane è un contorno tradizionale della regione Sicilia",
    "Il pistacchio di Bronte è coltivato in Sicilia ed è un prodotto DOP",
    "La cassata è un dolce tradizionale della regione Sicilia",

    # Emilia-Romagna
    "I tortellini in brodo sono il piatto simbolo della città di Bologna, in Emilia-Romagna",
    "Il Parmigiano Reggiano DOP è prodotto in Emilia-Romagna",
    "La mortadella IGP è un salume tipico della città di Bologna, in Emilia-Romagna",
    "I cappellacci di zucca sono un primo piatto tipico di Ferrara, in Emilia-Romagna",
    "L'aceto balsamico tradizionale DOP è prodotto a Modena, in Emilia-Romagna",
    "La piadina è una focaccia tipica della Romagna, in Emilia-Romagna",

    # Campania
    "La pizza Margherita è stata inventata a Napoli, in Campania",
    "La mozzarella di bufala DOP è prodotta in Campania",
    "Il babà al rum è un dolce tipico di Napoli, in Campania",
    "La sfogliatella riccia è un dolce tradizionale di Napoli, in Campania",
    "La pasta alla Genovese è un piatto tipico di Napoli, in Campania",
    "Il limoncello è un liquore prodotto nella costiera amalfitana, in Campania",

    # Veneto
    "Il baccalà alla vicentina è un piatto tipico di Vicenza, in Veneto",
    "Il risotto all'Amarone è un piatto della Valpolicella, in Veneto",
    "Il tiramisù è un dolce originario della città di Treviso, in Veneto",
    "I bigoli in salsa sono una pasta tipica di Venezia, in Veneto",
    "Il fegato alla veneziana è un piatto tradizionale di Venezia, in Veneto",
    "Il prosecco DOC è un vino prodotto in Veneto",

    # Piemonte
    "Il tartufo bianco d'Alba è un prodotto tipico del Piemonte",
    "I tajarin sono tagliolini all'uovo tipici del Piemonte",
    "Il brasato al Barolo è un secondo piatto tradizionale del Piemonte",
    "Il gianduiotto è un cioccolatino nato a Torino, in Piemonte",
    "La bagna càuda è una salsa tipica della tradizione piemontese",
    "Il Barolo è un vino DOCG prodotto in Piemonte"
]

class RAGSystem:
    def __init__(self, api_key):
        print("Inizializzazione del sistema RAG...")
        # Configurazione retrieval
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.sentences = knowledge_base
        self.embeddings = self.model.encode(self.sentences)

        # Configurazione Gemini
        genai.configure(api_key=api_key)

        # Configurazioni di generazione
        generation_config = {
            "temperature": 0.7,
            "max_output_tokens": 2048,
        }

        # Configurazioni di sicurezza
        safety_settings = [
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        ]

        # Inizializzazione modello Gemini
        self.llm = genai.GenerativeModel(
            model_name="gemini-1.5-flash",
            generation_config=generation_config,
            safety_settings=safety_settings
        )

        print("Sistema RAG inizializzato con successo!")

    def retrieve(self, query, k=3):
        """Recupera i k documenti più rilevanti"""
        query_embedding = self.model.encode([query])
        similarities = cosine_similarity(query_embedding, self.embeddings)[0]
        top_k_indices = similarities.argsort()[-k:][::-1]

        results = []
        for idx in top_k_indices:
            results.append({
                'text': self.sentences[idx],
                'similarity': similarities[idx]
            })
        return results

    def generate(self, query, retrieved_docs):
        """Genera una risposta usando Gemini"""
        # Costruisce il prompt con il contesto recuperato
        context = "\n".join([doc['text'] for doc in retrieved_docs])

        prompt = f"""
CONTESTO:
{context}

QUERY DELL'UTENTE:
{query}

ISTRUZIONI:
Genera una risposta alla domanda dell'utente utilizzando ESCLUSIVAMENTE le informazioni fornite nel contesto sopra.
Se alcune informazioni non sono presenti nel contesto, non inventarle.
Fornisci una risposta chiara e ben strutturata.
"""

        # Genera la risposta
        response = self.llm.generate_content(prompt)
        return response.text

    def query(self, user_query, k=3):
        """Processo RAG completo: retrieval + generation"""
        print("1. Recupero documenti rilevanti...")
        retrieved_docs = self.retrieve(user_query, k)

        print("2. Generazione risposta...")
        response = self.generate(user_query, retrieved_docs)

        return {
            'response': response,
            'retrieved_docs': retrieved_docs
        }

def print_full_results(results):
    """Stampa i risultati completi del sistema RAG"""
    print("\nDocumenti recuperati:")
    print("-" * 80)
    for i, doc in enumerate(results['retrieved_docs'], 1):
        print(f"{i}. [Score: {doc['similarity']:.3f}] {doc['text']}")

    print("\nRisposta generata:")
    print("-" * 80)
    print(results['response'])



/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:

# Assicurati di avere la tua API key in una variabile d'ambiente, la trovi su aistudio.google.com
api_key = "PASTEHEREYOURAPIKEY"
if not api_key:
    raise ValueError("Please set GOOGLE_API_KEY environment variable")

# Inizializza il sistema
rag = RAGSystem(api_key)



Inizializzazione del sistema RAG...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Sistema RAG inizializzato con successo!


In [3]:
# Esempi di query
test_queries = [
    "Quali sono i piatti tipici di Napoli?",
    "Dimmi dei prodotti DOP del Piemonte",
    "Che dolci famosi ci sono in Sicilia?",
    "Quali sono i piatti a base di pesce più famosi?"
]

# Testa il sistema
for query in test_queries:
    print(f"\nQuery: '{query}'")
    print("-" * 60)
    results = rag.query(query)
    print_full_results(results)


Query: 'Quali sono i piatti tipici di Napoli?'
------------------------------------------------------------
1. Recupero documenti rilevanti...
2. Generazione risposta...

Documenti recuperati:
--------------------------------------------------------------------------------
1. [Score: 0.794] La pasta alla Genovese è un piatto tipico di Napoli, in Campania
2. [Score: 0.709] I cappellacci di zucca sono un primo piatto tipico di Ferrara, in Emilia-Romagna
3. [Score: 0.708] Il babà al rum è un dolce tipico di Napoli, in Campania

Risposta generata:
--------------------------------------------------------------------------------
A Napoli, in Campania, sono tipici la pasta alla Genovese e il babà al rum.


Query: 'Dimmi dei prodotti DOP del Piemonte'
------------------------------------------------------------
1. Recupero documenti rilevanti...
2. Generazione risposta...

Documenti recuperati:
--------------------------------------------------------------------------------
1. [Score: 0.748] 

Proviamo anche una query "out of context"

In [7]:
result = rag.query("Come si chiamava il padre di Giuseppe Garibaldi?")
print(result['response'])

1. Recupero documenti rilevanti...
2. Generazione risposta...
Il contesto fornito non contiene informazioni sul padre di Giuseppe Garibaldi.

